Code for generating results for the semi-synthetic IHDP data (Figure 4(a) and Figure 4(b)).

In [ ]:
import numpy
import sympy
import pandas
import numpy as np
import pandas as pd
import sympy as sp
import datetime
import copy
import attr
import time
import logging
import itertools
import pickle
import sys
import os
import functools

import matplotlib.pyplot as plt
from matplotlib import rcParams
import collections

from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy import interpolate
import warnings
from IPython.display import clear_output

HOME_DIR = ""
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
class ModelParams:

  def __init__(self, beta, a1, a2, y):
    self.beta = beta
    self.a1 = a1
    self.a2 = a2
    self.y = y
  
  def get_true_param(self):
    return self.beta
  
  def __str__(self):
    return "b=%f, a1=%f, a2=%f, y=%f" % (self.beta, self.a1, self.a2, self.y)

truth = ModelParams(beta=1, a1=1, a2=.1, y=1)

In [ ]:
df_ihdp = pd.read_csv(os.path.join(HOME_DIR, "data/ihdp_processed.csv"))
print(df_ihdp.shape)
df_ihdp.head()

(747, 4)


,treat,bw,cig,bw_std
0,1,1559,0,-0.528957
1,0,1000,1,-1.738109
2,0,1430,0,-0.807992
3,0,1984,1,0.390344
4,0,1320,1,-1.045929


In [ ]:
def generate_data_samples(num_samples, model, k=[1/3, 1/3]):
  np = numpy
  pd = pandas
  
  u_y = np.random.normal(scale=np.sqrt(model.y), size=(num_samples,))
  df_idxs = np.argmax(np.random.multinomial(1, np.ones((df_ihdp.shape[0],)) / df_ihdp.shape[0],
                                            size=(num_samples,)), axis=-1)
  df = df_ihdp.iloc[df_idxs]

  def get_W1():
    return df["bw_std"].values
  
  def get_W2():
    return df["cig"].values
  
  def get_X():
    return df["treat"].values

  def get_Y(X, W1, W2, u):
    return model.beta * X + model.a1 * W1 + model.a2 * W2 + u
  
  W1 = get_W1()
  W2 = get_W2()
  X = get_X()
  Y = get_Y(X, W1, W2, u_y)
  SEL = np.zeros_like(Y, np.int32)

  SELS = [np.zeros_like(Y, np.int32) for _ in k]
  k_sum = 0
  for i, k_i in enumerate(k):
    prev_k_sum = k_sum
    k_sum += k_i
    SELS[i][int(prev_k_sum*Y.shape[0]) : int(k_sum*Y.shape[0])] = 1

  df = pd.DataFrame({
      "W1": W1,
      "W2": W2,
      "X": X,
      "Y": Y,
  })
  for i in range(len(k)):
    df["SEL%d" % (i+1)] = SELS[i]
  return df

In [ ]:
df = generate_data_samples(num_samples=10000, model=truth, k=[1/3, 1/3])
df.head()

,W1,W2,X,Y,SEL1,SEL2
0,-1.045929,0,0,-0.759767,1,0
1,-1.456911,0,0,-0.313307,1,0
2,0.424953,0,1,0.309768,1,0
3,-0.981037,1,0,0.101697,1,0
4,-0.483533,0,1,-0.076938,1,0


In [ ]:
def get_fraction_cost(cost=[1, 1, 1], k=[1/3, 1/3]):
  total = 0
  for c, k_val in zip(cost, k):
    total += c*k_val
  total += cost[-1]*(1 - sum(k))
  return total

In [ ]:
class GMMEqs:

  def __init__(self):
    self.moment_eqs, self.jacobian_eqs, self.moment_reweighting = self._get_equations()
  
  def get(self):
    return self.moment_eqs, self.jacobian_eqs, self.moment_reweighting
  
  def _get_equations(self):
    sp = sympy

    df_symbols = sp.symbols('ns1, ns2, s3, S1, S2, W1, W2, X, Y')
    ns1, ns2, s3, S1, S2, W1, W2, X, Y = df_symbols
    params = sp.symbols("b, a1, a2, t1, t2, d, w1, w2, y")
    b, a1, a2, t1, t2, d, w1, w2, y = params

    m_wts = [ns2, ns2, ns1, ns1, s3, ns2, ns1, ns2, ns1, ns2, ns1]
    moment_reweighting = sp.zeros(len(m_wts), len(m_wts))
    for i, mw1 in enumerate(m_wts):
      for j, mw2 in enumerate(m_wts):
        if mw1 == 1 or mw2 == 1:
          moment_reweighting[i, j] = mw1 * mw2
        elif mw1 == mw2:
          moment_reweighting[i, j] = mw1

    m1 = m_wts[0] * (1-S2) * (W1 * (Y - a1*W1 - b*X) - a2*d)
    m2 = m_wts[1] * (1-S2) * (X * (Y - a1*W1 - b*X) - a2*t2)
    m3 = m_wts[2] * (1-S1) * (W2 * (Y - a2*W2 - b*X) - a1*d)
    m4 = m_wts[3] * (1-S1) * (X * (Y - a2*W2 - b*X) - a1*t1)
    m5 = m_wts[4] * (1-S1-S2) * (W1*W2 - d)
    m6 = m_wts[5] * (1-S2) * (X*W1 - t1)
    m7 = m_wts[6] * (1-S1) * (X*W2 - t2)
    m8 = m_wts[7] * (1-S2) * (W1**2 - w1)
    m9 = m_wts[8] * (1-S1) * (W2**2 - w2)
    m10 = m_wts[9] * (1-S2) * ((Y - a1*W1 - b*X)**2 - a2**2 * w2 - y)
    m11 = m_wts[10] * (1-S1) * ((Y - a2*W2 - b*X)**2 - a1**2 * w1 - y)

    all_symbols = df_symbols + params
    moments = [m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11]
    jacobian = []
    for mom in moments:
      jac_row = []
      for p in params:
        eq = sp.simplify(sp.diff(mom, p))
        jac_row.append(sp.lambdify(all_symbols, eq, "numpy"))
      jacobian.append(jac_row)
    
    moments = [sp.lambdify(all_symbols, eq, "numpy") for eq in moments]
    moment_reweighting = sp.lambdify((ns1, ns2, s3), moment_reweighting, "numpy")
    return moments, jacobian, moment_reweighting

class GMM:

  def __init__(self, df, gmm_eqs):
    np = numpy

    # self.df = df
    self.df_dict = {k: df[k].values for k in df.columns}
    self.df = self.df_dict
    self.moment_eqs, self.jacobian_eqs, self.moment_reweighting = gmm_eqs.get()
    self.num_samples = len(df)
    self.num_moments = len(self.moment_eqs)
    self.momconds_arr = np.zeros((self.num_moments, self.num_samples))
  
  def momconds(self, params):
    """Returns the emprical moment vector (equivalent of \hat{g}_n(\theta))."""
    np = numpy

    n = self.num_samples
    return np.sum(self._momconds_arr(params), axis=-1) / n
  
  def _compute_moment_covariance(self, params):
    """Estimate the optimal weight matrix using the emprical moments."""
    n = self.num_samples
    moms = self._momconds_arr(params)
    moment_covariance = (moms @ moms.T) / n
    return moment_covariance
  
  def _get_objective_fn(self, weight_matrix_inv):
    """Returns the GMM objective function."""
    np = numpy
    
    w_lambda = 0.001
    weight_matrix_inv += weight_matrix_inv + w_lambda * np.eye(weight_matrix_inv.shape[0]) 

    def objective(params):
      moms = self.momconds(params)

      w_inv_mom = np.linalg.solve(weight_matrix_inv, moms)
      obj = moms.T @ w_inv_mom

      return obj
    
    return objective

  def _momconds_arr(self, params):
    np = numpy

    p = params
    df = self.df_dict
    n = self.num_samples

    for i, eq in enumerate(self.moment_eqs):
      self.momconds_arr[i, :] = eq(S1=df["SEL1"], S2=df["SEL2"],
                                   W1=df["W1"], W2=df["W2"],
                                   X=df["X"], Y=df["Y"],
                                   ns1=1, ns2=1, s3=1,
                                   b=p[0], a1=p[1], a2=p[2], t1=p[3], t2=p[4], d=p[5], w1=p[6], w2=p[7], y=p[8])
    
    return self.momconds_arr

  def _get_asymptotic_variance_matrix(self, k, moment_covariance, params):
    np = numpy

    current_k = [np.mean(self.df["SEL1"]), np.mean(self.df["SEL2"])]

    p = params
    df = self.df
    n = self.num_samples

    jacobian = np.zeros((self.num_moments, len(p)))

    ns1_rw = (1 - k[0]) / (1 - current_k[0])
    ns2_rw = (1 - k[1]) / (1 - current_k[1])
    s3_rw = (1-k[0]-k[1])/(1-current_k[0]-current_k[1])

    for i, jac_row in enumerate(self.jacobian_eqs):
      for j, eq in enumerate(jac_row):
        jacobian[i, j] = np.sum(eq(S1=df["SEL1"], S2=df["SEL2"],
                                   W1=df["W1"], W2=df["W2"],
                                   X=df["X"], Y=df["Y"],
                                   ns1=ns1_rw, ns2=ns2_rw, s3=s3_rw,
                                   b=p[0], a1=p[1], a2=p[2], t1=p[3], t2=p[4],
                                   d=p[5], w1=p[6], w2=p[7], y=p[8])) / n

    moment_covariance_reweight = self.moment_reweighting(ns1=ns1_rw, ns2=ns2_rw, s3=s3_rw,)
    
    moment_covariance = moment_covariance * moment_covariance_reweight
    return np.linalg.inv(jacobian.T @ np.linalg.inv(moment_covariance) @ jacobian)
  
  def _get_asymptotic_variance(self, k, moment_covariance, params):
    return self._get_asymptotic_variance_matrix(k, moment_covariance, params)[0][0]
  
  def _optimize_find_parameters(self, weight_matrix_inv, initial_guess=None):
    np = numpy

    initial_guess = np.array([1.1, 1, 1, 1, 1, 1, 1, 1, 1]) if initial_guess is None else initial_guess
    res = minimize(self._get_objective_fn(weight_matrix_inv), initial_guess,
                   bounds=[
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (-np.inf, np.inf),
                           (0.1, np.inf),
                           (0.1, np.inf),
                           (0.1, np.inf),
                   ],
                  )
    return res.x
  
  def find_parameters(self, num_iters=2, weight_matrix_reg=None):
    np = numpy

    weight_matrix_inv = np.eye(self.num_moments)
    for i in range(num_iters):
      params = self._optimize_find_parameters(weight_matrix_inv, initial_guess=params if i > 0 else None)
      weight_matrix_inv = self._compute_moment_covariance(params)

      if weight_matrix_reg is not None:
        weight_matrix_inv += weight_matrix_reg * np.eye(weight_matrix_inv.shape[0])
      
    return params, weight_matrix_inv

  def find_optimal_k(self, moment_covariance, params, cost=[1, 1, 1]):
    np = numpy
    
    initial_guess = np.array([1/3, 1/3])
    lower_bound = 0.05
    upper_bound = 0.95

    if np.sum(cost) == len(cost):
      objective = lambda x: self._get_asymptotic_variance(x, moment_covariance, params)
    else:
      objective = lambda x: self._get_asymptotic_variance(x, moment_covariance, params) * (get_fraction_cost(cost, x))
    res = minimize(objective,
                   initial_guess,
                   constraints=[LinearConstraint([1, 1], [1-upper_bound], [1-lower_bound]),
                                LinearConstraint([1, 0], [lower_bound], [upper_bound]),
                                LinearConstraint([0, 1], [lower_bound], [upper_bound])],
                   )
    
    if res.x[0] == lower_bound and res.x[1] == lower_bound:
      return np.array([0, 0])
    
    return res.x

In [ ]:
gmm = GMM(df, GMMEqs())
params, moment_covariance = gmm.find_parameters(num_iters=2)

print("True params:")
print(truth)
print("Estimated params:")
print(params)

optimal_k = gmm.find_optimal_k(moment_covariance, params)
print("Estimated optimal selection ratio k: %s" % str(optimal_k))

del gmm, params, moment_covariance, optimal_k

True params:
b=1.000000, a1=1.000000, a2=0.100000, y=1.000000
Estimated params:
[ 0.96694478  1.00800977  0.19191733  0.04045318  0.07599427 -0.06614089
  1.01743196  0.36492049  0.99095177]
Estimated optimal selection ratio k: [0.05 0.05]


In [ ]:
class SampleRevealer:

  def __init__(self, budget, cost, df):
    self.initial_budget = budget
    self.budget = budget
    self.cost = cost
    self.buffer_size = len(df)
    self.df = df
    self.counter = 0

  def reset(self):
    self.counter = 0
    self.budget = self.initial_budget
  
  def is_budget_left(self, samples_to_reveal, k):
    return self.budget >= samples_to_reveal * get_fraction_cost(self.cost, k)
  
  def reveal(self, reveal_k, samples_to_reveal):
    if self.counter + samples_to_reveal >= self.buffer_size:
      raise ValueError("no buffer")
    
    if not self.is_budget_left(samples_to_reveal, reveal_k):
      raise ValueError("no budget left")
    
    observe_count_1 = int(reveal_k[0] * samples_to_reveal)
    observe_count_2 = int(reveal_k[1] * samples_to_reveal)
    self.df["SEL1"][self.counter:self.counter+samples_to_reveal].values[:] = 0
    self.df["SEL2"][self.counter:self.counter+samples_to_reveal].values[:] = 0
    self.df["SEL1"].values[self.counter:self.counter+observe_count_1] = 1
    self.df["SEL2"].values[self.counter+observe_count_1:self.counter+observe_count_1+observe_count_2] = 1

    self.counter += samples_to_reveal
    self.budget -= (samples_to_reveal * get_fraction_cost(self.cost, reveal_k))
  
  def reveal_with_budget(self, reveal_k, budget_to_reveal):
    samples_to_reveal = int(budget_to_reveal / get_fraction_cost(self.cost, reveal_k))
    self.reveal(reveal_k, samples_to_reveal)
  
  def get_dataset(self):
    return self.df[:self.counter]

In [ ]:
def batch_fractions_to_sizes(horizon, batch_fractions):
  batch_sizes = []
    
  for i in range(len(batch_fractions) + 1):
    prev_batch_end = 0 if i == 0 else int(horizon * batch_fractions[i - 1])
    curr_batch_end = horizon if i == len(batch_fractions) else int(horizon * batch_fractions[i])

    batch_sizes.append(curr_batch_end - prev_batch_end)
  
  return batch_sizes

In [ ]:
def k_to_simplex(k):
  np = numpy

  k_simplex = np.zeros((len(k) + 1))
  k_simplex[:-1] = k
  k_simplex[-1] = 1 - np.sum(k)
  return k_simplex
  
def compute_reveal_k(current_samples, samples_to_reveal, current_k, target_k):
  np = numpy

  current_k = np.array(current_k)
  target_k = np.array(target_k)

  target_simplex = k_to_simplex(target_k)

  def objective(k):
    k_final = (current_k * current_samples + k * samples_to_reveal) / (current_samples + samples_to_reveal)
    diff = k_to_simplex(k_final) - target_simplex
    return np.dot(diff, diff)
  
  lower_bound = 0.01
  upper_bound = 0.99
  res = minimize(objective,
                 current_k,
                 constraints=[LinearConstraint([1, 1], [0], [1]),
                              LinearConstraint([1, 0], [lower_bound], [upper_bound]),
                              LinearConstraint([0, 1], [lower_bound], [upper_bound])],
                 )

  return res.x

def compute_reveal_k_with_budget(current_samples, budget_to_reveal, cost, current_k, target_k):
  np = numpy

  np = numpy

  current_k = np.array(current_k)
  target_k = np.array(target_k)

  target_simplex = k_to_simplex(target_k)

  def objective(k):
    samples_to_reveal = int(budget_to_reveal / get_fraction_cost(cost, k))
    k_final = (current_k * current_samples + k * samples_to_reveal) / (current_samples + samples_to_reveal)
    diff = k_to_simplex(k_final) - target_simplex
    return np.dot(diff, diff)

  lower_bound = 0.01
  upper_bound = 0.99
  res = minimize(objective,
                 current_k,
                 constraints=[LinearConstraint([1, 1], [0], [1]),
                              LinearConstraint([1, 0], [lower_bound], [upper_bound]),
                              LinearConstraint([0, 1], [lower_bound], [upper_bound])],
                 )

  return res.x

In [ ]:
class StrategyRunResult:

  def __init__(self):
    self.budgets_used = []
    self.squared_errors = []
    self.optimal_ks = []
    self.current_ks = []
  
  def append(self, budget_used, squared_error, optimal_k, current_k):
    self.budgets_used.append(budget_used)
    self.squared_errors.append(squared_error)
    self.optimal_ks.append(optimal_k)
    self.current_ks.append(current_k)

In [ ]:
class OracleStrategy:

  def __init__(self, sample_revealer, gmm_equations, optimal_k, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'oracle'
    self.gmm_equations = gmm_equations

    # Batches are budget fraction allocations.
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.optimal_k = optimal_k
    self.cost = cost
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    sel1 = np.sum(dataset["SEL1"])
    sel2 = np.sum(dataset["SEL2"])
    total = len(dataset)
    return np.array([sel1/total, sel2/total]), total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        current_k, current_samples = np.array([0, 0]), 0
      else:
        current_k, current_samples = self.get_current_df_vals()

      reveal_k = compute_reveal_k_with_budget(current_samples, batch_size, self.cost, current_k, self.optimal_k)

      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      _ = self.get_and_store_params()

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETCStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'etc'
    self.gmm_equations = gmm_equations
    # Batch sizes represent fractions of budget allocations.
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.cost = cost
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    sel1 = np.sum(dataset["SEL1"])
    sel2 = np.sum(dataset["SEL2"])
    total = len(dataset)
    return np.array([sel1/total, sel2/total]), total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = [0, 0]
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k_with_budget(current_samples, batch_size,
                                                self.cost, current_k, self.optimal_k)

      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      params = self.get_and_store_params()

      if i == 0:
        df = self.sample_revealer.get_dataset()
        if np.mean(df["X"]) <= 0.1:
          return None
        
        self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETGreedyFBStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    self.sample_revealer = sample_revealer
    self.name = 'etg-fb'
    self.gmm_equations = gmm_equations
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(budget, batch_fractions)
    self.cost = cost
    self.weight_matrix_reg = 0.01
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    sel1 = np.sum(dataset["SEL1"])
    sel2 = np.sum(dataset["SEL2"])
    total = len(dataset)
    return np.array([sel1/total, sel2/total]), total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self, is_last_step):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    if is_last_step:
      self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    else:
      self.params, self.moment_covariance = (
          self.gmm.find_parameters(num_iters=2, weight_matrix_reg=self.weight_matrix_reg)
      )
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = [0, 0]
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k_with_budget(current_samples, batch_size, self.cost, current_k, self.optimal_k)
      
      self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=batch_size)
      if i == 0:
        df = self.sample_revealer.get_dataset()
        if np.mean(df["X"]) <= 0.1:
          return None

      is_last_step = (i==len(self.batch_sizes)-1)
      params = self.get_and_store_params(is_last_step)

      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, _ = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
class ETGreedyFSStrategy:

  def __init__(self, sample_revealer, gmm_equations, cost, budget, batch_fractions):
    np = numpy

    self.sample_revealer = sample_revealer
    self.name = 'etg-fs'
    horizon_min = int(budget / np.max(cost))
    # We assume that the first batch size is exploration.
    self.batch_sizes = batch_fractions_to_sizes(horizon_min, batch_fractions)[:-1]
    self.cost = cost
    self.gmm_equations = gmm_equations
    self.weight_matrix_reg = 0.01
  
  def get_current_df_vals(self):
    np = numpy

    dataset = self.sample_revealer.get_dataset()

    sel1 = np.sum(dataset["SEL1"])
    sel2 = np.sum(dataset["SEL2"])
    total = len(dataset)
    return np.array([sel1/total, sel2/total]), total

  def can_step(self):
    return self.sample_revealer.is_budget_left()
  
  def get_and_store_params(self, is_last_step):
    dataset = self.sample_revealer.get_dataset()
    self.gmm = GMM(dataset, self.gmm_equations)
    if is_last_step:
      self.params, self.moment_covariance = self.gmm.find_parameters(num_iters=2)
    else:
      self.params, self.moment_covariance = (
          self.gmm.find_parameters(num_iters=2, weight_matrix_reg=self.weight_matrix_reg)
      )
    return self.params

  def get_squared_error(self):
    error = (truth.get_true_param() - self.params[0])**2
    return error
  
  def execute_run(self, cost=1):
    np = numpy

    result = StrategyRunResult()
    for i, batch_size in enumerate(self.batch_sizes):
      if i == 0:
        reveal_k = [0, 0]
      else:
        current_k, current_samples = self.get_current_df_vals()
        reveal_k = compute_reveal_k(current_samples, batch_size, current_k, self.optimal_k)

      self.sample_revealer.reveal(reveal_k=reveal_k, samples_to_reveal=batch_size)
      if i == 0:
        df = self.sample_revealer.get_dataset()
        if np.mean(df["X"]) <= 0.1:
          return None

      params = self.get_and_store_params(is_last_step=False)

      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, current_samples = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    last_step_reached = False
    batch_size = self.batch_sizes[-1]
    while not last_step_reached:      
      current_k, current_samples = self.get_current_df_vals()
      reveal_k = compute_reveal_k(current_samples, batch_size, current_k, self.optimal_k)

      if self.sample_revealer.is_budget_left(samples_to_reveal=batch_size, k=reveal_k):
        self.sample_revealer.reveal(reveal_k=reveal_k, samples_to_reveal=batch_size)
      else:
        budget_left = self.sample_revealer.budget
        reveal_k = compute_reveal_k_with_budget(current_samples, budget_left, self.cost, current_k, self.optimal_k)
        self.sample_revealer.reveal_with_budget(reveal_k=reveal_k, budget_to_reveal=budget_left)
        last_step_reached = True
      
      params = self.get_and_store_params(last_step_reached)
      self.optimal_k = self.gmm.find_optimal_k(self.moment_covariance, params, cost=cost)

      current_k, current_samples = self.get_current_df_vals()

      result.append(
          self.sample_revealer.initial_budget - self.sample_revealer.budget,
          self.get_squared_error(), self.optimal_k, current_k,
      )
    
    return result

In [ ]:
def execute_strategy_iteration(strategy_name, iteration_num, budget):
  np = numpy

  cost = [1, 3, 3.5]

  gmm_equations = GMMEqs()

  # Uncomment the next two lines to replicate the exact runs used in the paper.
  # random_seed = 232281293 + iteration_num
  # np.random.seed(random_seed)
  df = generate_data_samples(num_samples=budget * 5, model=truth)
  np.random.seed(None)

  sample_revealer = SampleRevealer(budget=budget, cost=cost, df=df)

  if strategy_name == "oracle":
    strategy = OracleStrategy(sample_revealer, gmm_equations=gmm_equations,
                              budget=budget, cost=cost, optimal_k=[0.59408789, 0],
                              batch_fractions=[0.1, 0.9])
  elif strategy_name == "collect_all":
    strategy = OracleStrategy(sample_revealer, gmm_equations=gmm_equations,
                              budget=budget, cost=cost, optimal_k=[0, 0],
                              batch_fractions=[0.1, 0.2, 0.5, 0.9])
  elif strategy_name == "etc_0.2":
    strategy = ETCStrategy(sample_revealer, gmm_equations=gmm_equations,
                           budget=budget, cost=cost,
                           batch_fractions=[0.2, 0.9])
  elif strategy_name == "etc_0.4":
    strategy = ETCStrategy(sample_revealer, gmm_equations=gmm_equations,
                           budget=budget, cost=cost,
                           batch_fractions=[0.4, 0.9])
  elif strategy_name == "etg-fb_0.4":
    strategy = ETGreedyFBStrategy(sample_revealer, gmm_equations=gmm_equations,
                                  budget=budget, cost=cost,
                                  batch_fractions=[0.4, 0.5, 0.6, 0.7, 0.8])
  elif strategy_name == "etg-fs_0.4":
    strategy = ETGreedyFSStrategy(sample_revealer, gmm_equations=gmm_equations,
                                  budget=budget, cost=cost,
                                  batch_fractions=[0.4, 0.8])
  else:
    raise ValueError("invalid strategy_name: %s" % strategy_name)

  return strategy.execute_run(cost=cost)

In [ ]:
# Test a strategy.

result = execute_strategy_iteration("etc_0.4", 1, budget=1000)
print(result.squared_errors)

[0.00555592535273502, 0.002185244501872844, 0.0024872469563182687]


**Executing the runs for each strategy in parallel.**

For the paper, we execute 12,000 runs for each strategy. We execute the runs
in parallel using
[`ipyparallel`](https://ipyparallel.readthedocs.io/en/latest/).

The following command starts the `ipyparallel` engines:
```
ipcluster start -n <num_engines>
```
 

In [ ]:
import ipyparallel as ipp

In [ ]:
# Verify that ipcluster is running and import the necessary Python packages.

parallel_client = ipp.Client(debug=False)
dview = parallel_client[:]
# Execute an identity map in parallel.
ar = dview.map(lambda x: x, (i for i in range(0, 2000000, 2)))
assert ar.get()[0] == 0

# Import the required Python packages.
with dview.sync_imports():
  from abc import ABC, abstractmethod
  import numpy
  import sympy
  import pandas
  import sympy
  import datetime
  import copy
  import attr
  import time
  import logging
  import itertools
  import pickle
  import os
  import functools
  import ipyparallel

  import collections

  from scipy.optimize import minimize
  from scipy import interpolate
  from scipy.special import expit
  from scipy.optimize import LinearConstraint
  import warnings

  try:
    from cPickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL
  except ImportError:
    from pickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL

# Make sure ipyparallel is still able to execute functions.
dview = parallel_client[:]
ar = dview.map(lambda x: x, (i for i in range(0, 2000000, 2)))
assert ar.get()[0] == 0

importing ABC,abstractmethod from abc on engine(s)
importing numpy on engine(s)
importing sympy on engine(s)
importing pandas on engine(s)
importing datetime on engine(s)
importing copy on engine(s)
importing attr on engine(s)
importing time on engine(s)
importing logging on engine(s)
importing itertools on engine(s)
importing pickle on engine(s)
importing os on engine(s)
importing functools on engine(s)
importing ipyparallel on engine(s)
importing collections on engine(s)
importing minimize from scipy.optimize on engine(s)
importing interpolate from scipy on engine(s)
importing expit from scipy.special on engine(s)
importing LinearConstraint from scipy.optimize on engine(s)
importing warnings on engine(s)


In [ ]:
def combine_parallel_results(async_result, need_interpolation=False):
  optimal_ks_res = []

  if not need_interpolation:
    budgets = async_result.get()[0].budgets_used
    errors = np.vstack([res.squared_errors for res in async_result.get() if res is not None])
    optimal_ks = np.vstack([[res.optimal_ks] for res in async_result.get() if res is not None])
    current_ks = np.vstack([[res.current_ks] for res in async_result.get() if res is not None])

    return budgets, errors, optimal_ks, current_ks
  
  budgets_matrix = [res.budgets_used for res in async_result.get() if res is not None]
  errors_matrix = [res.squared_errors for res in async_result.get() if res is not None]
  optimal_ks_matrix = [res.optimal_ks for res in async_result.get() if res is not None]
  current_ks_matrix = [res.current_ks for res in async_result.get() if res is not None]

  def find_budget():
    idx = 0
    target = budgets_matrix[0][-1]

    for i in range(1, len(budgets_matrix)):
      if target > budgets_matrix[i][-1]:
        target = budgets_matrix[i][-1]
        idx = i
    
    return np.array(budgets_matrix[idx])
  
  budgets = find_budget()
  errors = []
  optimal_ks = []
  current_ks = []

  for i in range(len(budgets_matrix)):
    f_err = interpolate.interp1d(budgets_matrix[i], errors_matrix[i])
    f_optimalk0 = interpolate.interp1d(budgets_matrix[i], np.array(optimal_ks_matrix[i])[:, 0])
    f_optimalk1 = interpolate.interp1d(budgets_matrix[i], np.array(optimal_ks_matrix[i])[:, 1])
    f_currentk0 = interpolate.interp1d(budgets_matrix[i], np.array(current_ks_matrix[i])[:, 0])
    f_currentk1 = interpolate.interp1d(budgets_matrix[i], np.array(current_ks_matrix[i])[:, 1])
    
    errors.append([])
    optimal_ks.append([])
    current_ks.append([])

    for b in budgets:
      errors[i].append(f_err(b))
      optimal_ks[i].append([f_optimalk0(b), f_optimalk1(b)])
      current_ks[i].append([f_currentk0(b), f_currentk0(b)])
  
  errors = np.array(errors)
  optimal_ks = np.array(optimal_ks)
  current_ks = np.array(current_ks)

  return budgets, errors, optimal_ks, current_ks

def execute_strategy_in_parallel(strategy_name, horizon, iterations):
  num_threads = len(parallel_client.ids)
  dview = parallel_client[:]

  print("Executing %s over %d iterations across %d cores" % (strategy_name, iterations, num_threads))

  dview["batch_fractions_to_sizes"] = batch_fractions_to_sizes
  dview["compute_reveal_k"] = compute_reveal_k
  dview["compute_reveal_k_with_budget"] = compute_reveal_k_with_budget
  dview["get_fraction_cost"] = get_fraction_cost
  dview["k_to_simplex"] = k_to_simplex
  dview["ModelParams"] = ModelParams
  dview["truth"] = truth
  dview["df_ihdp"] = df_ihdp
  dview["generate_data_samples"] = generate_data_samples
  dview["StrategyRunResult"] = StrategyRunResult
  dview["GMM"] = GMM
  dview["GMMEqs"] = GMMEqs
  dview["OracleStrategy"] = OracleStrategy
  dview["ETCStrategy"] = ETCStrategy
  dview["ETGreedyFBStrategy"] = ETGreedyFBStrategy
  dview["ETGreedyFSStrategy"] = ETGreedyFSStrategy
  dview["SampleRevealer"] = SampleRevealer
  dview["execute_strategy_iteration"] = execute_strategy_iteration

  def execute_iteration(i):
    return execute_strategy_iteration(strategy_name, i, horizon)

  return dview.map(execute_iteration, range(iterations))

In [ ]:
def get_timeseries_for(strategy_names, horizons, iterations, results_dict):
  results_dict["truth"] = truth

  for strategy_name in strategy_names:
    for horizon in horizons:
      print("Timestamp start: %s, Strategy: %s, Horizon: %d, Iters: %d" % (
          datetime.datetime.now(), strategy_name, horizon, iterations))
      async_result = execute_strategy_in_parallel(strategy_name, horizon, iterations)
      need_interpolation = "etg-fs" in strategy_name
      budgets, errors, optimal_ks, current_ks = combine_parallel_results(async_result, need_interpolation)
      print("Timestamp end: %s" % (datetime.datetime.now()))

      if strategy_name not in results_dict:
        results_dict[strategy_name] = {}
      
      results_dict[strategy_name][horizon] = {
          "budgets": budgets,
          "errors": errors * budgets,
          "optimal_ks": optimal_ks,
          "current_ks": current_ks,
      }

In [ ]:
results_dict = {}
get_timeseries_for([
                    "oracle", "collect_all",
                    "etc_0.2", "etc_0.4",
                    "etg-fb_0.4", "etg-fs_0.4",
                    ],
                   horizons=[500, 600, 700, 800, 900, 1000, 1150, 1300],
                   iterations=12*1000,
                   results_dict=results_dict)

In [ ]:
# Save results to file.
# pickle.dump(results_dict,
#             open(os.path.join(HOME_DIR,
#                               "%s_ihdp_graph.pkl" % datetime.datetime.now()),
#                  "wb"))

### Plot the results

In [ ]:
# For the color map:
# https://gist.github.com/AndiH/c957b4d769e628f506bd

# Tableau 20 Colors
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
             
# Tableau Color Blind 10
tableau20blind = [(0, 107, 164), (255, 128, 14), (171, 171, 171), (89, 89, 89),
             (95, 158, 209), (200, 82, 0), (137, 137, 137), (163, 200, 236),
             (255, 188, 121), (207, 207, 207)]
  
# Rescale to values between 0 and 1 
for i in range(len(tableau20)):  
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)
for i in range(len(tableau20blind)):  
    r, g, b = tableau20blind[i]  
    tableau20blind[i] = (r / 255., g / 255., b / 255.)
# Use with plt.plot(…, color=tableau[0],…)

In [ ]:
def plot_regret_curve(results_dict):

  clist = rcParams['axes.prop_cycle']
  cgen = itertools.cycle(clist)

  oracle_mses = {}
  for horizon, timeseries in results_dict["oracle"].items():
    oracle_mses[timeseries["budgets"][-1]] = np.mean(timeseries["errors"][:, -1])

  oracle_budgets = np.array(list(oracle_mses.keys()))

  def plot(axs, name, info, result, with_var=False):
    x_vals = []
    y_mean = []
    y_std = []

    current_k_mean = []
    optimal_k_mean = []

    for horizon, timeseries in result.items():
      x_vals.append(timeseries["budgets"][-1])

      closest_budget = oracle_budgets[np.argmin(np.abs(oracle_budgets - x_vals[-1]))]
      y_scaled = ( timeseries["errors"][:, -1] - oracle_mses[closest_budget] )  / oracle_mses[closest_budget] * 100
      y_mean.append(np.mean(y_scaled))
      y_std.append(np.std(y_scaled))

    x_vals = np.array(x_vals)
    y_mean = np.array(y_mean)
    y_std = np.array(y_std)
    
    x_sort_idx = np.argsort(x_vals)
    x_vals = x_vals[x_sort_idx]
    y_mean = y_mean[x_sort_idx]
    y_std = y_std[x_sort_idx]
    
    color = next(cgen)["color"]

    plt.plot(x_vals, y_mean, label=name, color=info[1], linestyle=info[0],  marker=info[2])
    plt.legend()

    if with_var:
      ci = 1.96 * y_std / np.sqrt(timeseries["errors"].shape[0])
      axs.errorbar(x_vals, y_mean, yerr=ci, ls="none", color=info[1])
      
  name_to_linestyle_color = {
      "etc_0.2": ["dashdot", tableau20blind[1], "s"],
      "etc_0.4": ["solid", tableau20blind[2], "v"],
      "etg-fb_0.4": ["solid", tableau20blind[8], "^"],
      "etg-fs_0.4": ["dashdot", tableau20blind[6], "D"],
      "collect_all": ["dashed", tableau20blind[7], "o"],
  }
  plt.title("Relative regret vs budget")
  plt.xlabel("Total budget ")
  plt.ylabel("Relative regret (%)")
  for name, info in name_to_linestyle_color.items():  
    plot(plt, name, info, results_dict[name], with_var=True)
  
  # plt.savefig(os.path.join(HOME_DIR, "figures/ihdp_regret_curve.eps"), bbox_inches='tight', pad_inches=0.0)

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE+4)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE+4)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE+20)  # fontsize of the figure title

In [ ]:
plot_regret_curve(results_dict)